In [1]:
import json
from pathlib import Path

from Libraries import Common_Utils as UTL
from Libraries import Common_Helpers as helpers
from Libraries import Processor_Datasets as ds_proc
from Libraries import Processor_Models as model_proc
from Libraries import Flow_Reasoning as flow_reason
from Libraries import Flow_Critical as flow_critic

In [2]:
# [0 - 999]
INDEX_START = 0
INDEX_END = 1

HISTORIES = Path("Data/Histories.json")
CONFIG_PATH = Path("Config/config.json")
BASE_MODELS_DIR = Path("Models")
BASE_DATA_DIR = Path("Data")

HISTORIES.parent.mkdir(parents=True, exist_ok=True)
BATCH_HISTORY_FILE = BASE_DATA_DIR / "Histories-Batch.json"

REASON_CLIENT = None
CRITIC_CLIENT = None
HF_DATASET = None
PROMPT_REASON_PATH = ""
PROMPT_CRITIC_PATH = ""
REASON_PROMPT = ""
CRITIC_PROMPT = ""
CONFIG = {}
GENERATION_PARAMS = {}
LLAMA_CPP_PARAMS = {}
FLOW_PARAMS = {}

CONFIG = UTL.read_json(CONFIG_PATH)

if CONFIG:
    PROMPT_REASON_PATH = Path(CONFIG["paths"]["prompt_reason"])
    PROMPT_REFINE_PATH = Path(CONFIG["paths"]["prompt_refine"])
    PROMPT_CRITIC_PATH = Path(CONFIG["paths"]["prompt_critic"])

    REASON_PROMPT = UTL.read_text(PROMPT_REASON_PATH)
    REFINE_PROMPT = UTL.read_text(PROMPT_REFINE_PATH)
    CRITIC_PROMPT = UTL.read_text(PROMPT_CRITIC_PATH)

    GENERATION_PARAMS = CONFIG.get("generation_params", {})
    LLAMA_CPP_PARAMS = CONFIG.get("llama_cpp_params", {})
    FLOW_PARAMS = CONFIG.get("flow_params", {})

    print("✅ Tất cả file cấu hình và prompt đã tải thành công.")

✅ Tất cả file cấu hình và prompt đã tải thành công.


In [3]:
if CONFIG:
    REASON_CLIENT, CRITIC_CLIENT = model_proc.llm_initialize(
        config=CONFIG,
        llama_cpp_params=LLAMA_CPP_PARAMS,
        base_models_dir=BASE_MODELS_DIR
    )

    dataset_config = CONFIG["dataset"]
    parts = dataset_config['name'].split('/')
    publisher = parts[0] if len(parts) > 1 else 'default'
    dataset_name = parts[-1]
    local_path = BASE_DATA_DIR / publisher / dataset_name
    
    if local_path.exists():
        print(f"✅ Đang tải dataset cục bộ từ: {local_path.resolve()}")
        HF_DATASET = ds_proc.load_from_disk_internal(local_path)
    else:
        print(f"⚠️ Không tìm thấy dataset cục bộ. Đang tải về từ '{dataset_config['name']}'...")
        HF_DATASET = ds_proc.download_and_save_internal(dataset_config, local_path)
        
    if not HF_DATASET:
            print("❌ Tải dataset thất bại.")
else:
    print("⛔ Config không được tải. Bỏ qua Giai đoạn 2.")

🔗 Using Local Llama HTTP server...
⏳ Đang kiểm tra Llama server tại http://localhost:8080...
✅ Server đã tải model và sẵn sàng (status: ok).
✅ Đang tải dataset cục bộ từ: E:\_DevResources\4. Python\Docs\Python--ReasCrit\Data\SurAyush\News_Summary_Dataset


In [4]:
if HF_DATASET:
    print("\n--- 📊 Phân tích Dataset ---")
    analysis = ds_proc.analyze_dataset_internal(HF_DATASET)
    if "error" in analysis:
        print(f"Lỗi phân tích: {analysis['error']}")
    else:
        print(f"Số lượng mẫu tổng: {analysis['count']}")
        print(f"Cấu trúc (Features): {analysis['features']}")
        print(f"Tìm thấy 'article': {analysis['has_article']}")
        print(f"Tìm thấy 'summary': {analysis['has_summary']}")
    print("."*50)

    if INDEX_END >= analysis['count']:
        print(f"⚠️ Cảnh báo: INDEX_END ({INDEX_END}) vượt quá số lượng mẫu ({analysis['count']}).")
        print(f"Sẽ tự động điều chỉnh INDEX_END về {analysis['count'] - 1}.")
        INDEX_END = analysis['count'] - 1
    
    print(f"\n✅ Sẵn sàng chạy quy trình cho các mẫu từ {INDEX_START} đến {INDEX_END}.")

else:
    print("⛔ Dataset chưa được tải, không thể tiếp tục.")


--- 📊 Phân tích Dataset ---
Số lượng mẫu tổng: 1000
Cấu trúc (Features): {'article': Value('string'), 'summary': Value('string')}
Tìm thấy 'article': True
Tìm thấy 'summary': True
..................................................

✅ Sẵn sàng chạy quy trình cho các mẫu từ 0 đến 1.


In [ ]:
import sys

def mainFlow(source_text: str, max_iters: int, min_improve: float) -> dict:
    
    best_reasoning = None
    best_score = 0.0
    history_log = {
        "source_text": source_text,
        "iterations": []
    }
    current_feedback = None
    current_reasoning = ""

    for step in range(1, max_iters + 1):

        print(f"\n🔄 Vòng {step} ...")
        
        current_reasoning = flow_reason.run(
            client=REASON_CLIENT,
            reason_prompt=REASON_PROMPT,
            refine_prompt=REFINE_PROMPT,
            generation_params=GENERATION_PARAMS,
            current_reasoning=current_reasoning,
            text=source_text,
            feedback=current_feedback
        )
        
        # Notebook xử lý lỗi
        if not current_reasoning:
            print("⛔ Lỗi: Reasoning trả về rỗng, dừng vòng lặp.")
            break
            
        print(f"""\n🔄 Reaoning Result:\n{current_reasoning}""")
        
        # --- GỌI MODULE CRITICAL ---
        critical_output = flow_critic.run(
            client=CRITIC_CLIENT,
            critic_prompt=CRITIC_PROMPT,
            source_text=source_text,
            reasoning_output=current_reasoning
        )
        
        # Notebook xử lý lỗi
        if "error" in critical_output:
            print(f"⛔ Lỗi từ Critical: {critical_output['error']}")
            if "raw_response" in critical_output:
                print("--- RAW OUTPUT ---")
                print(critical_output["raw_response"])
                print("------------------")
            history_log["iterations"].append({"round": step, "error": critical_output})
            break
        
        # --- GỌI MODULE HELPER ---
        average_score = helpers.average_score(critical_output)
        current_feedback = critical_output.get("feedback_text", "") 

        # Notebook in kết quả
        print(f"📊 Điểm TBC: {average_score:.2f}")
        print(f"📝 Nhận xét (Toàn bộ JSON): {json.dumps(critical_output, ensure_ascii=False, indent=2)}\n")

        # Log lại
        history_log["iterations"].append({
            "round": step, 
            "article:": source_text,
            "reasoning": current_reasoning, 
            "evaluation": critical_output.get("scoring", {}),
            "feedback": current_feedback
        })

        # So sánh điểm
        if (average_score - best_score) > min_improve:
            best_reasoning, best_score = current_reasoning, average_score
            print(f"📈 Cải thiện tốt, điểm mới: {best_score:.2f}")
        else:
            if step == 1:
                best_reasoning, best_score = current_reasoning, average_score
            print("⛔ Không cải thiện đáng kể, dừng.")
            break

    return {"best_reasoning": best_reasoning, "best_score": best_score, "history": history_log}

In [6]:
# ✅ PATCH CELL: CHẠY BATCH SAFE-SKIP
from Libraries.exceptions import PipelineAbortSample

successful_runs = 0

print(f"🚀 BẮT ĐẦU CHẠY HÀNG LOẠT CHO {INDEX_END - INDEX_START + 1} MẪU... ({INDEX_START} → {INDEX_END})")
print(f"Kết quả → {BATCH_HISTORY_FILE.resolve()}")
print("="*70)

for i in range(INDEX_START, INDEX_END + 1):
    print(f"\n\n--- 🔄 Mẫu #{i} ---")

    input_article = ds_proc.get_content_by_index_internal(HF_DATASET, i)
    if not input_article:
        print(f"⛔ Không lấy được article cho index {i}")
        continue

    try:
        print(f"Bắt đầu mainFlow → {FLOW_PARAMS}")
        result = mainFlow(
            input_article,
            max_iters=FLOW_PARAMS.get("max_iters", 3),
            min_improve=FLOW_PARAMS.get("min_improve", 0.05)
        )

        history_key = f"index_{i}"
        history_data = result["history"].get("iterations", [])

        UTL.update_json_dict(history_key, history_data, BATCH_HISTORY_FILE, indent=2)
        print(f"✅ Lưu xong {history_key}")
        successful_runs += 1

        print("\n" + "."*50)
        print(f"🎯 Điểm: {result['best_score']}")
        print(f"🧠 Tóm tắt: {result['best_reasoning']}")
        print("."*50)

    except PipelineAbortSample as e:
        print(f"⚠️ Skip mẫu {i}: {e}")
        UTL.update_json_dict(f"index_{i}", {"status":"skipped","error":str(e)}, BATCH_HISTORY_FILE, indent=2)
        continue

    except Exception as e:
        print(f"❌ Fatal lỗi tại index {i}: {e}")
        UTL.update_json_dict(f"index_{i}", {"status":"fatal","error":str(e)}, BATCH_HISTORY_FILE, indent=2)
        raise

print("\n" + "="*70)
print("✅ HOÀN TẤT")
print(f"🎉 Thành công: {successful_runs} mẫu")
print(f"📦 Log: {BATCH_HISTORY_FILE.resolve()}")


🚀 BẮT ĐẦU CHẠY HÀNG LOẠT CHO 2 MẪU... (0 → 1)
Kết quả → E:\_DevResources\4. Python\Docs\Python--ReasCrit\Data\Histories-Batch.json


--- 🔄 Mẫu #0 ---
Bắt đầu mainFlow → {'max_iters': 3, 'min_improve': 0.05}

🔄 Vòng 1 ...

🔄 Reaoning Result:
REASONING:
Topic: US consumer confidence
Key ideas: Economy, optimism, job creation, economic growth, retail sales
Filtered ideas: Economic expansion, job growth, consumer spending, retail sales, Federal Reserve interest rates
SUMMARY: US consumer confidence peaked, driven by economic expansion, job growth, and retail sales. Optimism for 2005, with strong post-Christmas sales, suggests continued economic growth. The Federal Reserve raised interest rates five times since June due to the robust economy....
📊 Điểm TBC: 9.83
📝 Nhận xét (Toàn bộ JSON): {
  "scoring": {
    "factuality": 10,
    "clarity": 9,
    "logical_coherence": 10,
    "coverage": 10,
    "utility": 10,
    "consistency": 10
  },
  "feedback_text": "The summary accurately captures t